by wedad\
Hey team this file contains the optimisation for logistic Regression Optimzation and experimentations\
I'll be comparing them to the baseline model found in baseline_model.ipynb\
The columns dropped are selected from '4_FeatureEngineeringRegression' file

# LogisticRegression Optimisation
Approaches I took were feature selection, Hyperparam tuning, Different logistic Model testing\
And Ensemble of different models\
Overall the best performing model was the Ensemble with reduced feature set, as see below:
--------------------------------------------



Read in all the relevant Libraries


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
import warnings
warnings.filterwarnings('ignore')

In [2]:
seed_list = [1,2,3,4,5,6,7,8,9,10]

Read In the data

In [3]:
aki_df = pd.read_csv('../data/df_final_AKI.csv')
aki_df = aki_df.drop(columns=['Unnamed: 0', 'subject_id'])
aki_df

,dod,gender,age,Albumin,Creatinine,Hemoglobin,INR(PT),PT,Sodium,Urea Nitrogen,Arterial Blood Pressure diastolic,Arterial Blood Pressure systolic,Heart Rate,Respiratory Rate,hypertension,chronic_kidney_disease,sepsis,Intercept
0,0,1,73,3.6,1.2,9.4,3.2,13.8,138.0,16.0,48.0,101.0,84.0,20.0,1,0,0,1
1,0,0,24,4.1,0.5,10.2,1.2,15.7,143.0,6.0,56.0,114.0,59.0,16.0,0,0,0,1
2,0,0,54,4.1,0.4,10.2,1.0,14.4,142.0,22.0,48.0,130.0,59.0,24.0,1,0,0,1
3,0,1,84,3.5,1.1,10.2,1.6,18.7,144.0,17.0,56.0,114.0,89.0,21.0,1,0,0,1
4,0,0,59,3.6,0.6,10.2,1.7,12.4,153.0,8.0,70.0,91.0,99.0,21.0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9804,0,1,61,3.3,3.6,10.2,1.5,13.1,138.0,23.0,62.0,103.0,89.0,20.0,0,1,0,1
9805,0,1,74,3.5,0.6,10.2,1.2,13.6,138.0,6.0,56.0,114.0,107.0,22.0,1,0,0,1
9806,0,1,58,4.1,0.6,10.2,0.9,9.3,141.0,8.0,56.0,114.0,78.0,10.0,0,0,0,1
9807,1,1,84,4.0,3.4,12.2,1.1,17.9,141.0,88.0,56.0,114.0,101.0,25.0,1,1,0,1


In [4]:
model_desc = []
model_comparison_DF = pd.DataFrame(model_desc, columns=['model', 'accuracy', 'precision', 'recall', 'F1 score'])
model_comparison_DF

,model,accuracy,precision,recall,F1 score


The function below takes in different Linear Models, and tests them over number of different Seeds to get an Average perfromance

In [5]:
x = aki_df.drop(columns=['dod'])
y = aki_df['dod']

def logisticTester(model_desc, model, x, y, title='alg', printer=False):
    total_acc = 0
    total_pre = 0
    total_rec = 0
    total_f1 = 0
    total = len(seed_list)
    for rand in seed_list:
        x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.25, random_state=rand)
        model.fit(x_train, y_train)
        y_pred_log = model.predict(x_test)
        total_acc += accuracy_score(y_test, y_pred_log)
        total_pre += precision_score(y_test, y_pred_log)
        total_rec += recall_score(y_test, y_pred_log)
        total_f1 += f1_score(y_test, y_pred_log)

    model_desc.append([
        title,
        total_acc/total,
        total_pre/total,
        total_rec/total,
        total_f1/total
        
                      ])
    if printer:
        # Evaluate
        print(f"{title}")
        print('--------------------------------------------')
        print("Accuracy:", total_acc/total,)
        print("Precision: ", total_pre/total,)
        print("recall: ", total_rec/total,)
        print("f1_score: ", total_f1/total,)
        print('--------------------------------------------')
        # print("Classification Report:\n", classification_report(y_test, y_pred_bin))
        plt.show()
    

In [6]:
log_reg = LogisticRegression()
logisticTester(model_desc, log_reg, x, y,'Logistic Regression baseline')

Baseline model, accuracy of approximately 0.844\
It is important that due to the imbalance\ 
Predicting every value to be 0 gives 83% accuracy\
In our case, it is important that we focus on Precision and Recall

In [7]:
x = aki_df.drop(columns=['dod','Sodium','Hemoglobin','gender','hypertension','Intercept'])
y = aki_df['dod']
x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.25, random_state=seed_list[0])

In [8]:
logisticTester(model_desc, log_reg, x, y,'Logistic Regression modified Features')

Baseline model reduced feature, accuracy of approximately 0.846\

In [9]:
# ‘lbfgs’ - [‘l2’, None]
# ‘newton-cholesky’ - [‘l2’, None]
# ‘newton-cg’ - [‘l2’, None]
# ‘sag’ - [‘l2’, None]
# ‘saga’ - [‘elasticnet’, ‘l1’, ‘l2’, None]
# ‘liblinear’ - [‘l1’, ‘l2’]
# ‘saga’ - [‘elasticnet’, ‘l1’, ‘l2’, None]

c_val = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]  # Regularization strength
c_weight = [None, 'balanced']



print("It takes 10 mins to run this code below")
# f1_score max | model -> lbfg C:0.0001 weight: balanced penalty: l2 | Accuracy: precision: 0.713168 0.329235 Recall: 0.669157 F1 Score: 0.441194
# Accuracy max | model -> sag C:10 weight: None penalty: None | Accuracy: 0.845862  precision: 0.679856  Recall: 0.169157 F1 Score: 0.270599

print("------------L2 and None Penalty -------------")
penalty_list = ['l2', None]

for c in c_val:
    for weight in c_weight:
        for pen in penalty_list:
            log_lbgfs = LogisticRegression(penalty=pen, solver= 'lbfgs', class_weight = weight, max_iter=1000, C=c)
            title = f'lbfg C:{c} weight: {weight} penalty: {pen}'
            logisticTester(model_desc, log_lbgfs, x, y, title, printer=False)
            
            log_n_chol = LogisticRegression(penalty=pen, solver= 'newton-cholesky', class_weight = weight, max_iter=1000, C=c)
            title = f'newton-cholesky C:{c} weight: {weight} penalty: {pen}'
            logisticTester(model_desc, log_n_chol, x, y, title, printer=False)
            
            log_n_cg = LogisticRegression(penalty=pen, solver= 'newton-cg', class_weight = weight, max_iter=1000, C=c)
            title = f'log_n_cg C:{c} weight: {weight} penalty: {pen}'
            logisticTester(model_desc, log_n_cg, x, y, title, printer=False)
            
            log_sag = LogisticRegression(penalty=pen, solver= 'sag', class_weight = weight, max_iter=1000, C=c)
            title = f'sag C:{c} weight: {weight} penalty: {pen}'
            logisticTester(model_desc, log_sag, x, y, title, printer=False)
            
            log_saga = LogisticRegression(penalty=pen, solver= 'saga', class_weight = weight, max_iter=1000, C=c)
            title = f'saga C:{c} weight: {weight} penalty: {pen}'
            logisticTester(model_desc, log_saga, x, y, title, printer=False)
            
print("It takes 6 mins to run this code below")            
print("------------L1 Penalty -------------")
penalty_list = ['l1']

for c in c_val:
    for weight in c_weight:
        for pen in penalty_list:
            log_liblinear = LogisticRegression(penalty=pen, solver= 'liblinear', class_weight = weight, max_iter=1000, C=c)
            title = f'liblinear C:{c} weight: {weight} penalty: {pen}'
            logisticTester(model_desc, log_liblinear, x, y, title, printer=False)
            
            log_saga = LogisticRegression(penalty=pen, solver= 'saga', class_weight = weight, max_iter=1000, C=c)
            title = f'saga C:{c} weight: {weight} penalty: {pen}'
            logisticTester(model_desc, log_saga, x, y, title, printer=False)

print("It takes 3 mins to run this code below")
print("------------elasticnet Penalty -------------")
penalty_list = ['elasticnet']

for c in c_val:
    for weight in c_weight:
        for pen in penalty_list:   
            log_saga = LogisticRegression(penalty=pen, solver= 'saga', class_weight = weight, max_iter=1000, C=c, l1_ratio=0.5)
            title = f'saga C:{c} weight: {weight} penalty: {pen}'
            logisticTester(model_desc, log_saga, x, y, title, printer=False)
            
   


It takes 20 mins to run this code below
------------L2 and None Penalty -------------
It takes 6 mins to run this code below
------------L1 Penalty -------------
It takes 3 mins to run this code below
------------elasticnet Penalty -------------


In [13]:
model_comparison_DF = pd.DataFrame(model_desc, columns=['model', 'accuracy', 'precision', 'recall', 'F1 score'])
model_comparison_DF


,model,accuracy,precision,recall,F1 score
0,Logistic Regression baseline,0.844109,0.655321,0.166988,0.265826
1,Logistic Regression modified Features,0.845577,0.673793,0.170361,0.271598
2,lbfg C:0.0001 weight: None penalty: l2,0.839503,0.707442,0.089880,0.159130
3,newton-cholesky C:0.0001 weight: None penalty: l2,0.839503,0.707442,0.089880,0.159130
4,log_n_cg C:0.0001 weight: None penalty: l2,0.839503,0.707442,0.089880,0.159130
...,...,...,...,...,...
179,saga C:1 weight: balanced penalty: elasticnet,0.711741,0.327431,0.666988,0.439134
180,saga C:10 weight: None penalty: elasticnet,0.845618,0.681981,0.164337,0.264540
181,saga C:10 weight: balanced penalty: elasticnet,0.711863,0.327594,0.667229,0.439336
182,saga C:100 weight: None penalty: elasticnet,0.845618,0.681981,0.164337,0.264540


In [20]:
model_comparison_DF.sort_values(by=['F1 score'], ascending=False).head(2)

,model,accuracy,precision,recall,F1 score
72,lbfg C:0.1 weight: balanced penalty: l2,0.713861,0.329974,0.669639,0.441970
73,newton-cholesky C:0.1 weight: balanced penalty...,0.713942,0.330014,0.669398,0.441949


In [21]:
model_comparison_DF.sort_values(by=['accuracy'], ascending=False).head(2)

,model,accuracy,precision,recall,F1 score
154,liblinear C:0.1 weight: None penalty: l1,0.846433,0.695005,0.165301,0.266840
65,sag C:0.1 weight: None penalty: l2,0.846311,0.689264,0.167470,0.269204


In [22]:
model_comparison_DF.sort_values(by=['precision'], ascending=False).head(2)

,model,accuracy,precision,recall,F1 score
26,saga C:0.001 weight: None penalty: l2,0.844476,0.772868,0.115181,0.200306
25,sag C:0.001 weight: None penalty: l2,0.844476,0.772868,0.115181,0.200306


In [23]:
model_comparison_DF.sort_values(by=['recall'], ascending=False).head(2)

,model,accuracy,precision,recall,F1 score
72,lbfg C:0.1 weight: balanced penalty: l2,0.713861,0.329974,0.669639,0.441970
112,lbfg C:10 weight: balanced penalty: l2,0.713127,0.329256,0.669398,0.441254


Ensemble of 4 best performing modesl for Accuracym Precision, Recall and F1 score


Description | Model | Accuracy | Precision | Recall | F1 Score\
best f1: lbfg C:0.1 weight: balanced penalty: l2	0.713861	0.329974	0.669639	0.441970\
best accuracy: liblinear C:0.1 weight: None penalty: l1	0.846433	0.695005	0.165301	0.266840\
best precision: saga C:0.001 weight: None penalty: l2	0.844476	0.772868	0.115181	0.200306\
brest recall: lbfg C:0.1 weight: balanced penalty: l2	0.713861	0.329974	0.669639	0.441970

In [40]:
def logisticEnsemble(model_list, model_desc, x, y, title='Combined Ensemble'):
    prediction_list = []
    total_acc = 0
    total_pre = 0
    total_rec = 0
    total_f1 = 0
    total = len(seed_list)
    
    for rand in seed_list:
        prediction_list = []
        for model in model_list:
            x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.25, random_state=rand)
            model.fit(x_train, y_train)
            y_pred_log = model.predict(x_test)
            prediction_list.append(y_pred_log)

        prediction_list = [sum(x) for x in zip(*prediction_list)]
        final_prediction = []
        for pred in prediction_list:
            if pred >1:
                final_prediction.append(1)
            else:
                final_prediction.append(0)
                
        total_acc += accuracy_score(y_test, final_prediction)
        total_pre += precision_score(y_test, final_prediction)
        total_rec += recall_score(y_test, final_prediction)
        total_f1 += f1_score(y_test, final_prediction)


    model_desc.append([
        "logistic Regression Ensemble",
        total_acc/total,
        total_pre/total,
        total_rec/total,
        total_f1/total
                      ])
    
    # Evaluate
    print(f"{title}")
    print('--------------------------------------------')
    print("Accuracy:", total_acc/total)
    print("Precision: ", total_pre/total)
    print("recall: ", total_rec/total)
    print("f1_score: ", total_f1/total)
    print('--------------------------------------------')
    # print("Classification Report:\n", classification_report(y_test, y_pred_bin))
    plt.show()

In [43]:
x = aki_df.drop(columns=['dod','Sodium','Hemoglobin','gender','hypertension','Intercept'])
y = aki_df['dod']

log_lbgfs = LogisticRegression(penalty='l2', solver= 'lbfgs', class_weight = 'balanced', max_iter=1000, C=0.1)
log_liblinear = LogisticRegression(penalty='l1', solver= 'liblinear', class_weight = None, max_iter=1000, C=0.1)
log_saga = LogisticRegression(penalty='l2', solver= 'saga', class_weight = None, max_iter=1000, C=0.001)

model_list = [log_lbgfs, log_liblinear, log_saga]

logisticEnsemble(model_list, model_desc, x, y, title='Reduced Feature Set Logistic Combined Ensemble')


x = aki_df.drop(columns=['dod'])
y = aki_df['dod']

logisticEnsemble(model_list, model_desc, x, y, title='Full Feature Set Logistic Combined Ensemble')

Reduced Feature Set Logistic Combined Ensemble
--------------------------------------------
Accuracy: 0.8473298002445985
Precision:  0.6938018224153757
recall:  0.17590361445783131
f1_score:  0.28038677574593573
--------------------------------------------
Full Feature Set Logistic Combined Ensemble
--------------------------------------------
Accuracy: 0.8465552384834896
Precision:  0.6756449129175457
recall:  0.17951807228915664
f1_score:  0.28345060616105877
--------------------------------------------
